In [ ]:
pip install mlflow

In [ ]:
# Use the exclamation mark to run shell commands from a Jupyter notebook
!mlflow ui

In [ ]:
pip install -r requirement.txt

In [ ]:
# Correct the SQLite URI format
!mlflow ui --backend-store-uri sqlite:///mlflow.db


In [2]:
!nohup mlflow ui

nohup: ignoring input and appending output to 'nohup.out'


In [ ]:
#homework
import pandas as pd
df_jan=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
len(df_jan.columns)


In [ ]:
#2. Now let's compute the duration variable. It should contain the duration of a ride in minutes.
#What's the standard deviation of the trips duration in January?

df_jan['duration']=df_jan.tpep_dropoff_datetime-df_jan.tpep_pickup_datetime
df_jan['duration'] = df_jan.duration.dt.total_seconds() / 60
df_jan.duration.std()

In [ ]:
#3. we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records 
# where the duration was between 1 and 60 minutes (inclusive).
# What fraction of the records left after you dropped the outliers?
total_records_before = len(df_jan)
total_records_before

In [ ]:
df_jan_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].copy()
total_records_after = len(df_jan_filtered)
total_records_after

In [ ]:
fraction_left = total_records_after / total_records_before
print(f"Fraction of records left after dropping outliers: {fraction_left:.2f}")

In [ ]:
#4.
#Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
#Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
#Fit a dictionary vectorizer
#Get a feature matrix from it

from sklearn.feature_extraction import DictVectorizer

df_jan
categorical = ['PULocationID','DOLocationID']

#change to object
df_jan[categorical] = df_jan[categorical].astype(str)

#change to dictionary
train_dicts = df_jan[categorical].to_dict(orient='records')


dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape


In [ ]:
#5.Now let's use the feature matrix from the previous step to train a model.

#Train a plain linear regression model with default parameters, where duration is the response variable
#Calculate the RMSE of the model on the training data
#What's the RMSE on train?

from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error

target='duration'
y_train=df_jan[target].values

lr=LinearRegression()
lr.fit(X_train,y_train)

y_pred=lr.predict(X_train)
mean_squared_error(y_train,y_pred,squared=False)

In [ ]:
#6. Now let's apply this model to the validation dataset (February 2023).
#What's the RMSE on validation?


categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].astype('str')
    return df

df_val = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
import mlflow
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set the MLflow tracking URI and experiment
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("second")

# Load the data
train_data_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
valid_data_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

df_train = pd.read_parquet(train_data_path)
df_valid = pd.read_parquet(valid_data_path)

# Feature engineering (example features)
df_train['duration'] = (df_train['tpep_dropoff_datetime'] - df_train['tpep_pickup_datetime']).dt.total_seconds() / 60
df_valid['duration'] = (df_valid['tpep_dropoff_datetime'] - df_valid['tpep_pickup_datetime']).dt.total_seconds() / 60

# Drop rows with duration outliers
df_train = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60)]
df_valid = df_valid[(df_valid['duration'] >= 1) & (df_valid['duration'] <= 60)]

# Example target and feature columns
target_col = 'duration'
feature_cols = ['passenger_count', 'trip_distance']

X_train = df_train[feature_cols]
y_train = df_train[target_col]
X_val = df_valid[feature_cols]
y_val = df_valid[target_col]

alpha = 0.01

# Run MLflow experiment
with mlflow.start_run():
    mlflow.set_tag("developer", "sandeep")
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", valid_data_path)
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)


OSError: Background processes not supported.